In [3]:
from telethon import TelegramClient, errors
import asyncio
import logging
import json
import os
import re
import sqlite3  # Import sqlite3 to handle database errors

from telethon.sessions import SQLiteSession

# Set up logging to see what's happening in real-time
logging.basicConfig(level=logging.INFO)

api_id = 26205886  # Your API ID
api_hash = '78566b82e3bf847041fe85a9d53b2d97'
phone_number = '+251939836983'  

# Initialize SQLiteSession
session = SQLiteSession('Ethio Mart')
client = TelegramClient(session, api_id, api_hash)

# Define output file for storing messages
output_file = 'fetched_messages.json'

# List to store fetched messages
all_messages = []

# Function to clean and preprocess Amharic text
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = text.strip()  # Remove leading/trailing spaces
    return text

async def fetch_messages(channel):
    retries = 5  # Number of retries
    for attempt in range(retries):
        try:
            async for message in client.iter_messages(channel, limit=100):  # Fetch 100 messages
                message_data = {
                    'channel': channel,
                    'sender': message.sender_id,
                    'timestamp': message.date.isoformat(),
                    'text': preprocess_text(message.text) if message.text else None,
                    'media': message.media,
                }

                # Log the message text if it exists
                if message_data['text'] is not None:  # Check if text is not None
                    logging.info(f"Fetched message from {channel}: {message_data['text'][:30]}...")  # Log the first 30 characters

                
                if message.media:
                    message_data['media'] = await handle_media(message.media)

                all_messages.append(message_data) 
            
            break  
        except errors.FloodWaitError as e:
            logging.warning(f"Rate limited. Retry in {e.seconds} seconds.")
            await asyncio.sleep(e.seconds)  
            break  
        except sqlite3.OperationalError as e:  
            logging.warning(f"Database locked. Attempt {attempt + 1} of {retries}. Retrying in 2 seconds...")
            await asyncio.sleep(2) 
        except Exception as e:
            logging.error(f"Error fetching messages: {e}")
            break  

async def handle_media(media):
    
    if media:
        try:
            if hasattr(media, 'photo'):
                
                file_path = f'images/{media.photo.id}.jpg'  
                await client.download_media(media, file_path)  
                return file_path
            elif hasattr(media, 'document'):
                
                file_path = f'documents/{media.document.id}.pdf'  
                await client.download_media(media, file_path)  
                return file_path
        except Exception as e:
            logging.error(f"Error handling media: {e}")
    return None

async def save_to_file():
    
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(all_messages, f, ensure_ascii=False, indent=4)
    logging.info(f"All messages saved to {output_file}")

async def main():
    await client.start(phone=phone_number) 

   
    verification_code = input("Please enter the code you received: ")
    await client.sign_in(phone_number, verification_code)  

    channels = [
        '@ZemenExpress',
        '@sinayelj',
        '@MerttEka',
        '@yebatochmregagroup',
        '@helloomarketethiopia',
        '@Leyueqa',
        '@kstoreaddis',
        '@Fashiontera'
    ]
    
    for channel in channels:
        await fetch_messages(channel)

    await save_to_file()  


os.makedirs('images', exist_ok=True)
os.makedirs('documents', exist_ok=True)


if __name__ == "__main__":
    try:
        
        loop = asyncio.get_event_loop()
        loop.run_until_complete(main())
    except RuntimeError as e:
        if 'This event loop is already running' in str(e):
           
            await main()  
        else:
            raise


INFO:telethon.network.mtprotosender:Connecting to 149.154.167.92:443/TcpFull...


INFO:telethon.network.mtprotosender:Connection to 149.154.167.92:443/TcpFull complete!
INFO:root:Fetched message from @ZemenExpress: 💥💥...............................
INFO:telethon.client.downloads:Starting direct file download in chunks of 131072 at 0, stride 131072
INFO:root:Fetched message from @ZemenExpress: 💥💥...............................
INFO:telethon.client.downloads:Starting direct file download in chunks of 131072 at 0, stride 131072
INFO:root:Fetched message from @ZemenExpress: 💥💥...............................
INFO:telethon.client.downloads:Starting direct file download in chunks of 131072 at 0, stride 131072
INFO:root:Fetched message from @ZemenExpress: 💥💥...............................
INFO:telethon.client.downloads:Starting direct file download in chunks of 131072 at 0, stride 131072
INFO:root:Fetched message from @ZemenExpress: 💥💥...............................
INFO:telethon.client.downloads:Starting direct file download in chunks of 131072 at 0, stride 131072
INFO:tel

ERROR:telethon.client.telegrambaseclient:Unhandled exception from keepalive_handle after cancelling <class '_asyncio.Task'> (<Task finished name='Task-44' coro=<UpdateMethods._keepalive_loop() done, defined at c:\Users\bam\AppData\Local\Programs\Python\Python312\Lib\site-packages\telethon\client\updates.py:481> exception=OperationalError('database is locked')>)
Traceback (most recent call last):
  File "c:\Users\bam\AppData\Local\Programs\Python\Python312\Lib\site-packages\telethon\helpers.py", line 176, in _cancel
    await task
  File "c:\Users\bam\AppData\Local\Programs\Python\Python312\Lib\site-packages\telethon\client\updates.py", line 517, in _keepalive_loop
    self._save_states_and_entities()
  File "c:\Users\bam\AppData\Local\Programs\Python\Python312\Lib\site-packages\telethon\client\telegrambaseclient.py", line 693, in _save_states_and_entities
    self.session.process_entities(types.contacts.ResolvedPeer(None, [e._as_input_peer() for e in entities], []))
  File "c:\Users\ba